# RusTorch CoreML 통합 - Rust 커널

이 노트북은 RusTorch와 CoreML을 사용하는 방법을 보여줍니다.
Rust 커널에서 실행됩니다.

## 필수 의존성 및 기능 확인

In [ ]:
// RusTorch 기본 사용
extern crate rustorch;

use rustorch::tensor::Tensor;
use rustorch::gpu::DeviceType;

println!("RusTorch 버전: {}", env!("CARGO_PKG_VERSION"));
println!("Rust 버전: {}", env!("RUSTC_VERSION"));

## CoreML 가용성 확인

현재 시스템에서 CoreML이 사용 가능한지 확인합니다.

In [ ]:
#[cfg(any(feature = "coreml", feature = "coreml-hybrid", feature = "coreml-fallback"))]
{
    use rustorch::backends::DeviceManager;
    
    let coreml_available = DeviceManager::is_coreml_available();
    println!("CoreML 사용 가능: {}", coreml_available);
    
    if coreml_available {
        println!("🎉 CoreML을 사용할 수 있습니다!");
        println!("플랫폼: macOS");
        
        // 디바이스 정보 표시
        use rustorch::gpu::coreml::device_cache::DeviceCache;
        let cache = DeviceCache::global();
        cache.warmup();
        
        let stats = cache.get_stats();
        println!("캐시 통계: {:?}", stats);
    } else {
        println!("⚠️ CoreML을 사용할 수 없습니다");
        println!("CPU 또는 다른 GPU 백엔드를 사용해주세요");
    }
}

#[cfg(not(any(feature = "coreml", feature = "coreml-hybrid", feature = "coreml-fallback")))]
{
    println!("❌ CoreML 기능이 활성화되지 않았습니다");
    println!("--features coreml로 빌드해주세요");
}

## 기본 텐서 연산

In [ ]:
// 기본 텐서 생성
let a = Tensor::zeros(&[2, 3]);
let b = Tensor::ones(&[3, 2]);

println!("텐서 A 형태: {:?}", a.shape());
println!("텐서 B 형태: {:?}", b.shape());

// 기본 행렬 곱셈
let result = a.matmul(&b);
println!("결과 형태: {:?}", result.shape());
println!("기본 텐서 연산 완료");

## CoreML 디바이스 연산

In [ ]:
#[cfg(any(feature = "coreml", feature = "coreml-hybrid", feature = "coreml-fallback"))]
{
    use rustorch::gpu::coreml::{CoreMLDevice, CoreMLBackend};
    use rustorch::backends::BackendConfig;
    
    // CoreML 디바이스 생성 시도
    match CoreMLDevice::new(0) {
        Ok(device) => {
            println!("🖥️ CoreML 디바이스 성공적으로 생성됨");
            println!("디바이스 ID: {}", device.id());
            println!("사용 가능: {}", device.is_available());
            println!("메모리 제한: {} MB", device.memory_limit() / (1024 * 1024));
            
            // 백엔드 구성 생성
            let config = BackendConfig::new()
                .with_caching(true)
                .with_max_cache_size(200)
                .with_profiling(true)
                .with_auto_fallback(true);
            
            println!("⚙️ 백엔드 구성: {:?}", config);
            
            // CoreML 백엔드 생성
            match CoreMLBackend::new(device, config) {
                Ok(backend) => {
                    println!("🚀 CoreML 백엔드 초기화됨");
                    
                    // 통계 가져오기
                    let stats = backend.get_statistics();
                    println!("📊 백엔드 통계:");
                    println!("  총 연산 수: {}", stats.total_operations);
                    println!("  캐시 히트: {}", stats.cache_hits);
                    println!("  캐시 미스: {}", stats.cache_misses);
                    println!("  폴백 연산: {}", stats.fallback_operations);
                    
                    // CoreML에서 텐서 생성
                    let tensor_a = Tensor::randn(&[64, 64]).to_device(&backend);
                    let tensor_b = Tensor::randn(&[64, 64]).to_device(&backend);
                    
                    println!("📐 CoreML 디바이스에서 텐서 생성됨");
                    
                    // 행렬 곱셈 연산
                    let start = std::time::Instant::now();
                    let result = tensor_a.matmul(&tensor_b);
                    let duration = start.elapsed();
                    
                    println!("✅ 행렬 곱셈 완료");
                    println!("⏱️ 실행 시간: {:?}", duration);
                    println!("🎯 결과 형태: {:?}", result.shape());
                    
                    // 캐시 정리
                    backend.cleanup_cache();
                    println!("🧹 캐시 정리됨");
                }
                Err(e) => println!("❌ CoreML 백엔드 생성 오류: {:?}", e),
            }
        }
        Err(e) => {
            println!("❌ CoreML 디바이스 생성 오류: {:?}", e);
            println!("이 시스템에서 CoreML을 사용할 수 없을 수 있습니다");
        }
    }
}

#[cfg(not(any(feature = "coreml", feature = "coreml-hybrid", feature = "coreml-fallback")))]
{
    println!("⚠️ CoreML 연산 건너뜀 - 기능이 활성화되지 않음");
}

## 성능 비교

In [ ]:
use std::time::Instant;

fn benchmark_operations() {
    let sizes = vec![(64, 64), (128, 128), (256, 256), (512, 512)];
    
    println!("🏁 연산 벤치마킹:");
    println!("크기\t\tCPU (ms)\t선호 디바이스");
    println!("-" * 40);
    
    for (rows, cols) in sizes {
        // CPU에서 텐서 생성
        let a = Tensor::randn(&[rows, cols]);
        let b = Tensor::randn(&[cols, rows]);
        
        // CPU 시간 측정
        let start = Instant::now();
        let _result = a.matmul(&b);
        let cpu_duration = start.elapsed();
        
        // 선호 디바이스 결정
        let preferred_device = if rows * cols < 1000 {
            "CPU"
        } else if rows * cols < 10000 {
            "Metal GPU"
        } else {
            "CoreML"
        };
        
        println!("{}x{}\t\t{:.2}\t\t{}", 
                rows, cols, 
                cpu_duration.as_millis() as f64, 
                preferred_device);
    }
}

benchmark_operations();
println!("\n📝 참고: 디바이스 선택은 텐서 크기와 가용성을 기반으로 합니다");

## 스마트 디바이스 선택

In [ ]:
#[cfg(any(feature = "coreml", feature = "coreml-hybrid", feature = "coreml-fallback"))]
{
    use rustorch::backends::{DeviceManager, DeviceSelector};
    
    fn demonstrate_device_selection() {
        println!("🎯 스마트 디바이스 선택 시연:");
        
        let operations = vec![
            ("작은 곱셈", vec![16, 16], "CPU"),
            ("2D 컨볼루션", vec![32, 3, 224, 224], "CoreML"),
            ("행렬 변환", vec![128, 128], "Metal GPU"),
            ("큰 배치 연산", vec![512, 512], "CoreML"),
            ("벡터 계산", vec![1000], "CPU"),
        ];
        
        for (name, shape, preferred) in operations {
            println!("  {:<25} {:?} -> {}", name, shape, preferred);
            
            // 규칙 기반 선택 시뮬레이션
            let tensor_size: usize = shape.iter().product();
            let selected_device = match tensor_size {
                size if size < 1000 => DeviceType::Cpu,
                size if size < 50000 => DeviceType::MetalGpu,
                _ => {
                    if DeviceManager::is_coreml_available() {
                        DeviceType::CoreML
                    } else {
                        DeviceType::MetalGpu
                    }
                }
            };
            
            println!("    -> 선택된 디바이스: {:?}", selected_device);
        }
        
        println!("\n📝 선택 논리:");
        println!("  • < 1K 요소: CPU (최소 오버헤드)");
        println!("  • 1K-50K 요소: Metal GPU (균형)");
        println!("  • > 50K 요소: CoreML (최적화) 또는 Metal GPU (폴백)");
    }
    
    demonstrate_device_selection();
}

#[cfg(not(any(feature = "coreml", feature = "coreml-hybrid", feature = "coreml-fallback")))]
{
    println!("⚠️ 디바이스 선택 시연 건너뜀 - CoreML 기능 사용 불가");
}

## 고급 예제: 신경망 레이어

In [ ]:
fn simulate_neural_layer() {
    println!("🧠 신경망 레이어 시뮬레이션:");
    
    // 레이어 구성
    let batch_size = 32;
    let input_dim = 784;   // 28x28 MNIST
    let hidden_dim = 256;
    let output_dim = 10;   // 10개 클래스
    
    println!("📊 구성:");
    println!("  배치 크기: {}", batch_size);
    println!("  입력 차원: {}", input_dim);
    println!("  은닉 차원: {}", hidden_dim);
    println!("  출력 차원: {}", output_dim);
    
    // 텐서 생성
    let input = Tensor::randn(&[batch_size, input_dim]);
    let weight1 = Tensor::randn(&[input_dim, hidden_dim]);
    let weight2 = Tensor::randn(&[hidden_dim, output_dim]);
    
    println!("\n🔄 순전파:");
    
    // 순전파 시뮬레이션
    let start = Instant::now();
    
    // 레이어 1: 입력 -> 은닉
    let hidden = input.matmul(&weight1);
    println!("  ✅ 입력 -> 은닉: {:?}", hidden.shape());
    
    // ReLU 활성화 함수 (시뮬레이션)
    let activated = hidden.relu();
    println!("  ✅ ReLU 활성화 적용됨");
    
    // 레이어 2: 은닉 -> 출력
    let output = activated.matmul(&weight2);
    println!("  ✅ 은닉 -> 출력: {:?}", output.shape());
    
    let total_time = start.elapsed();
    
    println!("\n⏱️ 총 순전파 시간: {:?}", total_time);
    println!("🚀 예상 성능: {:.0} 샘플/초", 
             (batch_size as f64) / total_time.as_secs_f64());
    
    println!("\n📝 실제 구현에서는:");
    println!("  • 큰 행렬은 CoreML 사용");
    println!("  • 활성화는 Metal GPU 사용");
    println!("  • 작은 연산은 CPU에 유지");
}

simulate_neural_layer();

## 오류 처리 및 폴백

In [ ]:
fn demonstrate_fallback_behavior() {
    println!("🔄 폴백 동작 시연:");
    
    // CoreML에서 실패할 수 있는 연산 시뮬레이션
    let complex_tensor = Tensor::randn(&[100, 100]);
    
    println!("🎯 CoreML 연산 시도 중...");
    
    // 실제 구현에서는 다음과 같습니다:
    // match tensor.to_coreml() {
    //     Ok(coreml_tensor) => { /* CoreML 사용 */ },
    //     Err(_) => { /* Metal/CPU로 폴백 */ }
    // }
    
    let use_coreml = false; // CoreML 실패 시뮬레이션
    
    if use_coreml {
        println!("✅ CoreML 연산 성공");
    } else {
        println!("⚠️ CoreML 사용 불가, 폴백 사용");
        
        // Metal GPU로 폴백
        let start = Instant::now();
        let result = complex_tensor.matmul(&complex_tensor);
        let fallback_time = start.elapsed();
        
        println!("✅ 폴백 연산 완료");
        println!("⏱️ 폴백 시간: {:?}", fallback_time);
        println!("📐 결과 형태: {:?}", result.shape());
    }
    
    println!("\n📝 폴백 전략:");
    println!("  1. CoreML 시도 (최고 성능)");
    println!("  2. Metal GPU로 폴백 (양호한 호환성)");
    println!("  3. CPU 최종 폴백 (최대 호환성)");
}

demonstrate_fallback_behavior();

## 요약 및 다음 단계

In [ ]:
println!("📋 RusTorch CoreML 통합 요약 (Rust 커널):");
println!();
println!("✅ 시연된 기능:");
println!("  • CoreML 가용성 확인");
println!("  • 디바이스 생성 및 관리");
println!("  • 백엔드 구성");
println!("  • 기본 텐서 연산");
println!("  • 성능 벤치마킹");
println!("  • 스마트 디바이스 선택");
println!("  • 폴백 동작");
println!();
println!("🚧 개발 영역:");
println!("  • 완전한 CoreML 연산 구현");
println!("  • 메모리 전송 최적화");
println!("  • 확장된 텐서 타입 지원");
println!("  • 상세한 성능 프로파일링");
println!("  • ML 파이프라인과의 통합");
println!();
println!("🎯 권장 다음 단계:");
println!("  1. 사전 훈련된 CoreML 모델로 테스트");
println!("  2. 다른 백엔드와 성능 비교");
println!("  3. 특정 사용 사례에 최적화");
println!("  4. 프로덕션 애플리케이션에 배포");
println!();

#[cfg(any(feature = "coreml", feature = "coreml-hybrid", feature = "coreml-fallback"))]
{
    if rustorch::backends::DeviceManager::is_coreml_available() {
        println!("🎉 모든 CoreML 기능을 테스트할 수 있습니다!");
    } else {
        println!("⚠️ CoreML이 활성화되었지만 이 시스템에서는 사용할 수 없습니다");
    }
}

#[cfg(not(any(feature = "coreml", feature = "coreml-hybrid", feature = "coreml-fallback")))]
{
    println!("⚠️ 완전한 기능을 위해 CoreML 기능으로 빌드하세요");
}

println!("\n🚀 RusTorch로 고급 CoreML 개발 준비 완료!");